In [141]:
import pandas as pd
import os
import concurrent.futures
import numpy as np
from ipysheet import from_dataframe, to_dataframe

pd.set_option('display.max_rows', 1000)

# Load Metadata

In [142]:
summary_file = os.path.join("data", "summary")
summary = pd.read_csv(summary_file)

In [143]:
summary.name

0               ABRCharacter
1                  ABRDragon
2                  Abilities
3               AbilityGroup
4        AbilityLimitedGroup
5                Adventurers
6              AffinityBonus
7                        Art
8       CampaignQuestHeaders
9                  CargoTest
10             CastleStories
11          ChainCoAbilities
12           CharaLimitBreak
13               CoAbilities
14                    Comics
15                Conditions
16               Consumables
17          DailyRewardChest
18                   Dragons
19               DropRewards
20              EndeavorSets
21           EndeavorSetsRow
22          EndeavorSetsTest
23               EnemiesTest
24            EnemyPageLinks
25         EventQuestHeaders
26                    Events
27                Facilities
28                     Gifts
29               LoadingTips
30               MCMaterials
31                MCNodeType
32                   MCNodes
33        MaintenanceNotices
34           M

# Weapon Calculations

## Exploring Columns

In [144]:
weapon_tables = summary[(summary.name.str.contains("Weapon")) | (summary.name == "Materials")].sort_values("records_pulled",ascending=False)
weapon_tables

,name,records_pulled,start_time,end_time,duration,url,errors
53,Weaponsv1,1139,1.609973e+09,1.609973e+09,0.848851,https://dragalialost.gamepedia.com/Special:Car...,NaN
47,WeaponBodyBuildupGroup,1085,1.609973e+09,1.609973e+09,0.392264,https://dragalialost.gamepedia.com/Special:Car...,NaN
51,WeaponSkin,695,1.609973e+09,1.609973e+09,0.498649,https://dragalialost.gamepedia.com/Special:Car...,NaN
50,WeaponPassiveAbility,415,1.609973e+09,1.609973e+09,0.335351,https://dragalialost.gamepedia.com/Special:Car...,NaN
36,Materials,354,1.609973e+09,1.609973e+09,0.295614,https://dragalialost.gamepedia.com/Special:Car...,NaN
52,Weapons,262,1.609973e+09,1.609973e+09,0.605974,https://dragalialost.gamepedia.com/Special:Car...,NaN
48,WeaponBodyBuildupLevel,220,1.609973e+09,1.609973e+09,0.217645,https://dragalialost.gamepedia.com/Special:Car...,NaN
49,WeaponBodyRarity,6,1.609973e+09,1.609973e+09,0.229509,https://dragalialost.gamepedia.com/Special:Car...,NaN


In [145]:
wep_table_list = weapon_tables.name.to_list()

def get_file(table):
    filename = os.path.join("data", f"{table}.csv")
    return {table: pd.read_csv(filename)}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_file, table) for table in wep_table_list]
    weapon_dfs = [f.result() for f in futures]
    weapon_dfs = {k: v for d in weapon_dfs for k,v in d.items()}

weapon_dfs.keys()

dict_keys(['Weaponsv1', 'WeaponBodyBuildupGroup', 'WeaponSkin', 'WeaponPassiveAbility', 'Materials', 'Weapons', 'WeaponBodyBuildupLevel', 'WeaponBodyRarity'])

In [146]:
for table, df in weapon_dfs.items():
    print(f"{table}:\n{[i for i in df.columns if 'Id' in i]}\n")

Weaponsv1:
['Id', 'BaseId', 'FormId', 'TypeId', 'ElementalTypeId', 'VariationId', 'CraftNodeId', 'ParentCraftNodeId', 'CraftGroupId', 'MainWeaponId', 'AcquiredWeaponId1', 'AcquiredWeaponId2', 'AvailabilityId']

WeaponBodyBuildupGroup:
['Id', 'WeaponBodyBuildupGroupId', 'BuildupPieceTypeId', 'BuildupMaterialId1', 'BuildupMaterialId2', 'BuildupMaterialId3', 'BuildupMaterialId4', 'BuildupMaterialId5', 'BuildupMaterialId6', 'BuildupMaterialId7', 'BuildupMaterialId8', 'BuildupMaterialId9', 'BuildupMaterialId10']

WeaponSkin:
['Id', 'WeaponTypeId', 'ElementalTypeId', 'BaseId', 'VariationId', 'FormId', 'DecBaseId', 'DecVariationId', 'BulletBaseId', 'BulletVariationId']

WeaponPassiveAbility:
['Id', 'WeaponPassiveAbilityGroupId', 'WeaponTypeId', 'ElementalTypeId', 'SortId', 'AbilityId', 'RewardWeaponSkinId1', 'RewardWeaponSkinId2', 'UnlockMaterialId1', 'UnlockMaterialId2', 'UnlockMaterialId3', 'UnlockMaterialId4', 'UnlockMaterialId5']

Materials:
['Id', 'QuestEventId', 'SortId']

Weapons:
['Id

## Querying Weapons (Exploration)

In [147]:
df_weapons = weapon_dfs["Weapons"]
print(df_weapons.columns.sort_values())
df_weapons[df_weapons.WeaponSeries=='Agito'].sort_values(["WeaponType", "ElementalType"]).head()

Index(['Abilities11', 'Abilities12', 'Abilities13', 'Abilities21',
       'Abilities22', 'Abilities23', 'BaseAtk', 'BaseHp', 'ChangeSkillId1',
       'ChangeSkillId2', 'ChangeSkillId3', 'CreateCoin', 'CreateEntity1',
       'CreateEntity2', 'CreateEntity3', 'CreateEntity4', 'CreateEntity5',
       'CreateEntityQuantity1', 'CreateEntityQuantity2',
       'CreateEntityQuantity3', 'CreateEntityQuantity4',
       'CreateEntityQuantity5', 'CreateStartDate',
       'CreateStartDate__precision', 'CrestSlotType1BaseCount',
       'CrestSlotType1MaxCount', 'CrestSlotType2BaseCount',
       'CrestSlotType2MaxCount', 'DuplicateEntity', 'DuplicateEntityQuantity',
       'ElementalType', 'ElementalTypeId', 'Id', 'IsPlayable',
       'LimitOverCountPartyPower1', 'LimitOverCountPartyPower2', 'MaxAtk1',
       'MaxAtk2', 'MaxAtk3', 'MaxHp1', 'MaxHp2', 'MaxHp3', 'MaxLimitOverCount',
       'MaxWeaponPassiveCharaCount', 'Name', 'NameJP',
       'NeedAllUnlockWeaponBodyId1', 'NeedCreateWeaponBodyId1',
  

,Id,Name,NameJP,WeaponSeries,WeaponSeriesId,WeaponSkinId,WeaponType,WeaponTypeId,Rarity,ElementalType,...,RewardWeaponSkinId2,RewardWeaponSkinId3,RewardWeaponSkinId4,RewardWeaponSkinId5,_pageName,_pageTitle,_pageNamespace,_pageID,_ID,CreateStartDate__precision
117,30460101,Mjölnir,ミョルニル,Agito,4,30460101,Axe,4,6,Flame,...,0,0,0,0,Mjölnir,Mjölnir,0,15249,172,0
120,30460401,Rog Mol,ローグ・モル,Agito,4,30460401,Axe,4,6,Light,...,0,0,0,0,Rog Mol,Rog Mol,0,22392,238,0
121,30460501,Fangtian Huaji,方天画戟,Agito,4,30460501,Axe,4,6,Shadow,...,0,0,0,0,Fangtian Huaji,Fangtian Huaji,0,17005,198,0
118,30460201,Ohohagari,大葉刈,Agito,4,30460201,Axe,4,6,Water,...,0,0,0,0,Ohohagari,Ohohagari,0,20310,221,0
119,30460301,Marmyadose,マルミアドワーズ,Agito,4,30460301,Axe,4,6,Wind,...,0,0,0,0,Marmyadose,Marmyadose,0,18628,209,0


In [148]:
df_weaponCost = weapon_dfs["WeaponBodyBuildupGroup"]
print(df_weaponCost.columns)
df_weaponCost.head()

Index(['Id', 'WeaponBodyBuildupGroupId', 'BuildupPieceTypeId',
       'BuildupPieceType', 'Step', 'UnlockConditionLevel',
       'UnlockConditionLimitBreakCount', 'UnlockConditionLimitOverCount',
       'RewardWeaponSkinNo', 'BuildupCoin', 'BuildupMaterialId1',
       'BuildupMaterialQuantity1', 'BuildupMaterialId2',
       'BuildupMaterialQuantity2', 'BuildupMaterialId3',
       'BuildupMaterialQuantity3', 'BuildupMaterialId4',
       'BuildupMaterialQuantity4', 'BuildupMaterialId5',
       'BuildupMaterialQuantity5', 'BuildupMaterialId6',
       'BuildupMaterialQuantity6', 'BuildupMaterialId7',
       'BuildupMaterialQuantity7', 'BuildupMaterialId8',
       'BuildupMaterialQuantity8', 'BuildupMaterialId9',
       'BuildupMaterialQuantity9', 'BuildupMaterialId10',
       'BuildupMaterialQuantity10', '_pageName', '_pageTitle',
       '_pageNamespace', '_pageID', '_ID'],
      dtype='object')


,Id,WeaponBodyBuildupGroupId,BuildupPieceTypeId,BuildupPieceType,Step,UnlockConditionLevel,UnlockConditionLimitBreakCount,UnlockConditionLimitOverCount,RewardWeaponSkinNo,BuildupCoin,...,BuildupMaterialQuantity8,BuildupMaterialId9,BuildupMaterialQuantity9,BuildupMaterialId10,BuildupMaterialQuantity10,_pageName,_pageTitle,_pageNamespace,_pageID,_ID
0,10000101,1000,1,Unbind,1,0,0,0,0,1000,...,0,0,0,0,0,Template:WeaponBodyBuildupGroup/Data,WeaponBodyBuildupGroup/Data,10,24278,7387
1,10000102,1000,1,Unbind,2,0,0,0,0,2000,...,0,0,0,0,0,Template:WeaponBodyBuildupGroup/Data,WeaponBodyBuildupGroup/Data,10,24278,7388
2,10000103,1000,1,Unbind,3,0,0,0,0,3000,...,0,0,0,0,0,Template:WeaponBodyBuildupGroup/Data,WeaponBodyBuildupGroup/Data,10,24278,7389
3,10000104,1000,1,Unbind,4,0,0,0,0,4000,...,0,0,0,0,0,Template:WeaponBodyBuildupGroup/Data,WeaponBodyBuildupGroup/Data,10,24278,7390
4,11000101,1100,1,Unbind,1,0,0,0,0,1000,...,0,0,0,0,0,Template:WeaponBodyBuildupGroup/Data,WeaponBodyBuildupGroup/Data,10,24278,7391


In [149]:
df_materials = weapon_dfs["Materials"]
df_materials["Id"] = pd.to_numeric(df_materials["Id"], errors="coerce", downcast="integer")
print(df_materials.columns)
print(df_materials.dtypes)

Index(['Id', 'Name', 'Description', 'Rarity', 'QuestEventId', 'Category',
       'SortId', 'Obtain', 'Uses', 'MoveQuest1', 'MoveQuest2', 'MoveQuest3',
       'MoveQuest4', 'MoveQuest5', 'PouchRarity', 'Exp', 'MaterialType',
       '_pageName', '_pageTitle', '_pageNamespace', '_pageID', '_ID'],
      dtype='object')
Id                float64
Name               object
Description        object
Rarity            float64
QuestEventId      float64
Category           object
SortId             object
Obtain             object
Uses               object
MoveQuest1        float64
MoveQuest2        float64
MoveQuest3        float64
MoveQuest4        float64
MoveQuest5        float64
PouchRarity       float64
Exp               float64
MaterialType      float64
_pageName          object
_pageTitle         object
_pageNamespace      int64
_pageID             int64
_ID                 int64
dtype: object


In [150]:
print(df_weaponCost.dtypes)

Id                                 int64
WeaponBodyBuildupGroupId           int64
BuildupPieceTypeId                 int64
BuildupPieceType                  object
Step                               int64
UnlockConditionLevel               int64
UnlockConditionLimitBreakCount     int64
UnlockConditionLimitOverCount      int64
RewardWeaponSkinNo                 int64
BuildupCoin                        int64
BuildupMaterialId1                 int64
BuildupMaterialQuantity1           int64
BuildupMaterialId2                 int64
BuildupMaterialQuantity2           int64
BuildupMaterialId3                 int64
BuildupMaterialQuantity3           int64
BuildupMaterialId4                 int64
BuildupMaterialQuantity4           int64
BuildupMaterialId5                 int64
BuildupMaterialQuantity5           int64
BuildupMaterialId6                 int64
BuildupMaterialQuantity6           int64
BuildupMaterialId7                 int64
BuildupMaterialQuantity7           int64
BuildupMaterialI

In [151]:
weapon_calc_df = df_weapons.merge(df_weaponCost, how="left", left_on="WeaponBodyBuildupGroupId", right_on="WeaponBodyBuildupGroupId", suffixes=("", "_buildUp"), copy=True)
for col in [i for i in df_weaponCost.columns if "MaterialId" in i]:
    num = col.split("Id")[1]
    weapon_calc_df = weapon_calc_df.merge(df_materials.add_suffix(f"_{num}"), how="left", left_on=col, right_on=f"Id_{num}", suffixes=("", ""), copy=True)
weapon_calc_df.columns.tolist()

['Id',
 'Name',
 'NameJP',
 'WeaponSeries',
 'WeaponSeriesId',
 'WeaponSkinId',
 'WeaponType',
 'WeaponTypeId',
 'Rarity',
 'ElementalType',
 'ElementalTypeId',
 'Obtain',
 'MaxLimitOverCount',
 'BaseHp',
 'MaxHp1',
 'MaxHp2',
 'MaxHp3',
 'BaseAtk',
 'MaxAtk1',
 'MaxAtk2',
 'MaxAtk3',
 'LimitOverCountPartyPower1',
 'LimitOverCountPartyPower2',
 'CrestSlotType1BaseCount',
 'CrestSlotType1MaxCount',
 'CrestSlotType2BaseCount',
 'CrestSlotType2MaxCount',
 'ChangeSkillId1',
 'ChangeSkillId2',
 'ChangeSkillId3',
 'Abilities11',
 'Abilities12',
 'Abilities13',
 'Abilities21',
 'Abilities22',
 'Abilities23',
 'IsPlayable',
 'Text',
 'CreateStartDate',
 'NeedFortCraftLevel',
 'NeedCreateWeaponBodyId1',
 'NeedCreateWeaponBodyId2',
 'NeedAllUnlockWeaponBodyId1',
 'CreateCoin',
 'CreateEntity1',
 'CreateEntityQuantity1',
 'CreateEntity2',
 'CreateEntityQuantity2',
 'CreateEntity3',
 'CreateEntityQuantity3',
 'CreateEntity4',
 'CreateEntityQuantity4',
 'CreateEntity5',
 'CreateEntityQuantity5',
 '

In [152]:
filt = weapon_calc_df.WeaponSeries == "Agito"
agito_weapon_calc_df = weapon_calc_df[filt].sort_values(["Id", "Step"])[
    ["Name", 'WeaponSeries', 'WeaponType','Rarity','ElementalType','Step','BuildupPieceType']+[j for i in range(1,11) for j in [f"Name_{i}", f"BuildupMaterialQuantity{i}"]]
].replace(0, np.nan).dropna(axis=1, how="all")
agito_weapon_calc_df.head(16)

,Name,WeaponSeries,WeaponType,Rarity,ElementalType,Step,BuildupPieceType,Name_1,BuildupMaterialQuantity1,Name_2,BuildupMaterialQuantity2,Name_3,BuildupMaterialQuantity3,Name_4,BuildupMaterialQuantity4,Name_5,BuildupMaterialQuantity5
292,Nothung,Agito,Sword,6,Flame,1.0,Unbind,Destitute One's Mask Fragment,40.0,Plagued One's Mask Fragment,30.0,Twinkling Sand,1.0,NaN,NaN,NaN,NaN
301,Nothung,Agito,Sword,6,Flame,1.0,Refinement,Destitute One's Mask Fragment,16.0,Plagued One's Mask Fragment,10.0,Rebellious One's Insanity,7.0,Orichalcum,1.0,NaN,NaN
303,Nothung,Agito,Sword,6,Flame,1.0,Slots,Destitute One's Mask Fragment,16.0,Plagued One's Mask Fragment,10.0,Orichalcum,1.0,NaN,NaN,NaN,NaN
304,Nothung,Agito,Sword,6,Flame,1.0,Weapon Bonus,Destitute One's Mask Fragment,100.0,Plagued One's Mask Fragment,100.0,Rebellious One's Insanity,14.0,Orichalcum,30.0,NaN,NaN
293,Nothung,Agito,Sword,6,Flame,2.0,Unbind,Destitute One's Mask Fragment,40.0,Plagued One's Mask Fragment,30.0,Twinkling Sand,1.0,NaN,NaN,NaN,NaN
302,Nothung,Agito,Sword,6,Flame,2.0,Refinement,Destitute One's Mask Fragment,40.0,Plagued One's Mask Fragment,30.0,Rebellious One's Insanity,10.0,Rebellious Wolf's Gale,10.0,Orichalcum,10.0
305,Nothung,Agito,Sword,6,Flame,2.0,Copies,Destitute One's Mask Fragment,280.0,Plagued One's Mask Fragment,200.0,Rebellious One's Insanity,35.0,Twinkling Sand,5.0,Orichalcum,5.0
294,Nothung,Agito,Sword,6,Flame,3.0,Unbind,Destitute One's Mask Fragment,40.0,Plagued One's Mask Fragment,30.0,Twinkling Sand,1.0,NaN,NaN,NaN,NaN
306,Nothung,Agito,Sword,6,Flame,3.0,Copies,Destitute One's Mask Fragment,280.0,Plagued One's Mask Fragment,200.0,Rebellious One's Insanity,35.0,Twinkling Sand,5.0,Orichalcum,5.0
295,Nothung,Agito,Sword,6,Flame,4.0,Unbind,Destitute One's Mask Fragment,40.0,Plagued One's Mask Fragment,30.0,Twinkling Sand,1.0,NaN,NaN,NaN,NaN


# Extracting Data

In [153]:
test = agito_weapon_calc_df.head(16)
def get_materials(df):
    materials = pd.DataFrame(columns = ["ElementalType", "Name", "Quantity"])
    materials_col = [i for i in agito_weapon_calc_df.columns if not i.isalpha()]
    names_col = [i for i in materials_col if "_" in i]
    paired_cols = [materials_col[i:i+2] for i in range(0,len(materials_col)-1, 2)]
    materials = pd.concat([materials] + 
                          [df[["ElementalType"] + cols].rename(
                              columns={cols[0]:"Name", cols[1]: "Quantity"}) 
                           for cols in paired_cols]
                         ).dropna()
    return materials.groupby(["ElementalType","Name"]).sum().reset_index()
get_materials(test)

,ElementalType,Name,Quantity
0,Flame,Destitute One's Mask Fragment,1316.0
1,Flame,Orichalcum,81.0
2,Flame,Plagued One's Mask Fragment,980.0
3,Flame,Rebellious One's Insanity,194.0
4,Flame,Rebellious Wolf's Gale,30.0
5,Flame,Twinkling Sand,19.0


# UI

## Agito

In [154]:
try:
    indexed_df = pd.read_csv("my_agito_weapons_df.csv")
except Exception as e:
    print(e)
    elements = agito_weapon_calc_df.ElementalType.unique()
    types = agito_weapon_calc_df.WeaponType.unique()
    index = pd.MultiIndex.from_tuples([(wep_type, el) for wep_type in types for el in elements], names=["Weapon", "Element"])
    indexed_df = pd.DataFrame([{"Tier": -1, "Refined": False, "Bonus": False, "Slots": False} for i in index], index=index)
indexed_df

,Tier,Refined,Bonus,Slots
0,-1,False,False,False
1,-1,False,False,False
2,-1,False,False,False
3,-1,False,False,False
4,-1,False,False,False
5,-1,False,False,False
6,-1,False,False,False
7,-1,False,False,False
8,-1,False,False,False
9,-1,False,False,False


In [155]:
sheet = from_dataframe(indexed_df)
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=44, row_start=0, squeeze_row=F…

### Save and Compute

In [156]:
my_weapons_df = to_dataframe(sheet)
my_weapons_df.head()

,Tier,Refined,Bonus,Slots
0,-1,False,False,False
1,-1,False,False,False
2,-1,False,False,False
3,-1,False,False,False
4,-1,False,False,False


In [157]:
my_weapons_df.to_csv("my_agito_weapons_df.csv")

In [158]:
def get_type(df, weapon, element):
    return (df["ElementalType"] == element) & (df["WeaponType"] == weapon)


def get_remaining_tiers(df, tier):
    return (
        (df['BuildupPieceType'] != "Unbind") | 
        ((df["Step"] > tier) & (df["BuildupPieceType"] == "Unbind") & (df["Step"] < 9))
    )


def get_refinements_remaining(df, refined):
    other_steps = (df["BuildupPieceType"] != "Refinement")
    if refined:
        return other_steps
    return (other_steps | (df["BuildupPieceType"] == "Refinement") & (df["Step"] == 1))


def get_weapon_bonus_remaining(df, bonused):
    other_steps = (df["BuildupPieceType"] != "Weapon Bonus")
    if bonused:
        return other_steps
    return True

def get_filt(row):
    return (
        get_type(agito_weapon_calc_df, row[0], row[1]) & 
        get_remaining_tiers(agito_weapon_calc_df, row.Tier) & 
        get_refinements_remaining(agito_weapon_calc_df, row.Refined) &
        get_weapon_bonus_remaining(agito_weapon_calc_df, row.Bonus)
    )

def apply_my_weapons(row):
    return (get_materials(agito_weapon_calc_df[get_filt(row)]))

# Calculated Totals

## Agito

In [159]:
pd.concat(my_weapons_df.apply(apply_my_weapons, axis=1).values).groupby(
    ["Name"]).sum().reset_index().sort_values("Quantity", ascending=False)

KeyError: 'Name'

### Compare to Total Needed

#### Explore

In [135]:
agito_weapon_calc_df[get_filt(pd.Series({
    "Weapon":"Sword", 
    "Element":"Flame", 
    "Tier": 8, 
    "Refined": True, 
    "Bonus": False, 
    "Slots": False}))]

,Name,WeaponSeries,WeaponType,Rarity,ElementalType,Step,BuildupPieceType,Name_1,BuildupMaterialQuantity1,Name_2,BuildupMaterialQuantity2,Name_3,BuildupMaterialQuantity3,Name_4,BuildupMaterialQuantity4,Name_5,BuildupMaterialQuantity5
303,Nothung,Agito,Sword,6,Flame,1.0,Slots,Destitute One's Mask Fragment,16.0,Plagued One's Mask Fragment,10.0,Orichalcum,1.0,NaN,NaN,NaN,NaN
304,Nothung,Agito,Sword,6,Flame,1.0,Weapon Bonus,Destitute One's Mask Fragment,100.0,Plagued One's Mask Fragment,100.0,Rebellious One's Insanity,14.0,Orichalcum,30.0,NaN,NaN
305,Nothung,Agito,Sword,6,Flame,2.0,Copies,Destitute One's Mask Fragment,280.0,Plagued One's Mask Fragment,200.0,Rebellious One's Insanity,35.0,Twinkling Sand,5.0,Orichalcum,5.0
306,Nothung,Agito,Sword,6,Flame,3.0,Copies,Destitute One's Mask Fragment,280.0,Plagued One's Mask Fragment,200.0,Rebellious One's Insanity,35.0,Twinkling Sand,5.0,Orichalcum,5.0
307,Nothung,Agito,Sword,6,Flame,4.0,Copies,Destitute One's Mask Fragment,280.0,Plagued One's Mask Fragment,200.0,Rebellious One's Insanity,35.0,Twinkling Sand,5.0,Orichalcum,5.0
300,Nothung,Agito,Sword,6,Flame,9.0,Unbind,Destitute One's Mask Fragment,80.0,Plagued One's Mask Fragment,70.0,Rebellious One's Insanity,30.0,Rebellious Wolf's Gale,20.0,Orichalcum,20.0


#### Output

In [127]:
get_materials(agito_weapon_calc_df).groupby(
    ["ElementalType", "Name"]).sum().reset_index().sort_values("Quantity", ascending=False)

,ElementalType,Name,Quantity
0,Flame,Destitute One's Mask Fragment,11844.0
22,Wind,Eliminating One's Mask Fragment,10764.0
8,Light,Remaining One's Mask Fragment,10764.0
19,Water,Soaring Ones' Mask Fragment,10764.0
11,Shadow,Almighty One's Mask Fragment,9568.0
2,Flame,Plagued One's Mask Fragment,8820.0
21,Wind,Despairing One's Mask Fragment,7920.0
10,Light,Vengeful One's Mask Fragment,7920.0
16,Water,Liberated One's Mask Fragment,7920.0
15,Shadow,Uprooting One's Mask Fragment,7040.0
